In [58]:
from constans import CONTINOUS_COLUMNS,DROP_COLUMNS,DISCRETE_COLUMNS,NOMINAL_COLUMNS,TARGET
import pandas as pd
from MLModel import MLModel

In [59]:
model_handler = MLModel()

fill_values_nominal = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_nominal.pkl')
fill_values_discrete = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_discrete.pkl')
fill_values_continuous = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_continuous.pkl')
min_max_scaler_dict = model_handler.load_model('artifacts/encoders/min_max_scaler_dict.pkl')
onehot_encoders = model_handler.load_model('artifacts/encoders/onehot_encoders_dict.pkl')
xgb = model_handler.load_model('artifacts/models/xgb_model.pkl')

In [60]:
data_path = 'data/train_pipeline/car_price_prediction.csv'
df = pd.read_csv(data_path)

In [61]:
sample_data = df.iloc[0]
sample_data = pd.DataFrame([sample_data])

In [62]:
sample_data.drop(columns=DROP_COLUMNS, inplace=True)

for col in NOMINAL_COLUMNS:
    sample_data[col].fillna(fill_values_nominal[col],
                                    inplace=True)
for col in DISCRETE_COLUMNS:
    sample_data[col].fillna(fill_values_discrete[col],
                                    inplace=True)
for col in CONTINOUS_COLUMNS:
    sample_data[col].fillna(fill_values_continuous[col],
                                    inplace=True)


for col, encoder in onehot_encoders.items():
    new_data = encoder.transform(sample_data[col].to_numpy().reshape(-1, 1))
    new_df = pd.DataFrame(new_data, columns=encoder.get_feature_names([col]))
    sample_data = pd.concat([sample_data, new_df], axis=1).drop(columns=[col])


for col, scaler in min_max_scaler_dict.items():
    if col in sample_data.columns:
        sample_data[col] = scaler.transform(sample_data[[col]])

if TARGET in sample_data.columns:
    sample_data = sample_data.drop(columns=TARGET)

predicted_value = xgb.predict(sample_data)

print(f"The predicted value is: {predicted_value[0]}")

TypeError: 'NoneType' object is not subscriptable